In [ ]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd

# Parameters
ncontract = 14
smplStart = '1983M01'
smplEnd = '2017M12'

# Load data files
oil_data = sio.loadmat('oil_data_new.mat')
proxy = oil_data['proxy']

oil_surprises = sio.loadmat('OilSurprisesMLog.mat')
oilProxiesWTIM = oil_surprises['oilProxiesWTIM']
proxyRaw = oilProxiesWTIM[:, ncontract-1]

database = sio.loadmat('dataBaseM.mat')
data = database['data']
sampleDates = database['sampleDates']
sampleDatesNum = database['sampleDatesNum']

# Find sample indices
sampleDates_flat = sampleDates.flatten()
smplStartInd = np.where(sampleDates_flat == smplStart)[0][0]
smplEndInd = np.where(sampleDates_flat == smplEnd)[0][0]

# Extract sample data
data = data[smplStartInd:smplEndInd+1, :]
z = proxyRaw[smplStartInd:smplEndInd+1]
sampleDates = sampleDates[smplStartInd:smplEndInd+1, :]
sampleDatesNum = sampleDatesNum[smplStartInd:smplEndInd+1, :]

varNames_paper = ['Real oil price','Oil production','Oil stocks', 
                  'World Industrial Production', 'Industrial Production', 
                  'Consumer Price Index','Unemployment','Fed Funds Rate', 'Core CPI']

dataFrequency = 'M'

# Standardize the proxy using the standshock method
def standshock(x, noise=0.01):
    """
    Standardize a variable using the standshock method from Stata.
    Only standardizes non-zero elements, keeps zeros as zeros.
    """
    # Step 1: Get non-zero elements
    nonzero_mask = x != 0
    nonzero_only = x[nonzero_mask]
    
    # Step 2: Compute mean and std of non-zero elements
    mean_nonzero = np.mean(nonzero_only)
    std_nonzero = np.std(nonzero_only, ddof=1)  # Use sample std (ddof=1)
    
    # Step 3: Standardize non-zero elements
    standardized_temp = x.copy()
    standardized_temp[nonzero_mask] = (nonzero_only - mean_nonzero) / std_nonzero
    
    # Step 4: Rescale entire series to have std = 1
    final_std = np.std(standardized_temp, ddof=1)
    scaled = standardized_temp / final_std
    
    return scaled

z_std = standshock(z)

# Create DataFrame
df = pd.DataFrame(data, columns=varNames_paper[:data.shape[1]])
df['Oil_Proxy'] = z_std
df.head()

# ------------------------------------------------------------------
# 0.  Basic set-up
# ------------------------------------------------------------------
plt.style.use(['science', 'no-latex'])          # needs  pip install SciencePlots
alpha   = 0.10                                  # 90 % bands
zcrit   = 1.645                                 # N-(0,1) critical value
h_max   = 24

# ------------------------------------------------------------------
# 1.  Re-express levels as 100·log (leave UN, FFR, Oil_Proxy alone)
# ------------------------------------------------------------------
log_cols = ['Real oil price', 'Oil production', 'Oil stocks',
            'World Industrial Production', 'Industrial Production',
            'Consumer Price Index', 'Core CPI']
df[log_cols] = 100 * np.log(df[log_cols])

# ------------------------------------------------------------------
# 2.  First-differences (except Oil_Proxy) with short names
# ------------------------------------------------------------------
short = {'Real oil price':'price', 'Oil production':'prod',
         'Oil stocks':'stock', 'World Industrial Production':'WIP',
         'Industrial Production':'IP', 'Consumer Price Index':'CPI',
         'Unemployment':'UN', 'Fed Funds Rate':'FFR', 'Core CPI':'CCPI'}
for long, s in short.items():
    if long != 'Oil_Proxy':                     # keep Oil_Proxy in levels
        df[f'd_{s}'] = df[long].diff()

# ------------------------------------------------------------------
# 3.  Dummy variables & re-scale by coefficients
# ------------------------------------------------------------------
z = df['Oil_Proxy']
df['f1'] = -(z.between(-1.25, -0.01, inclusive='both')).astype(int)
df['f2'] = -(z < -1.25).astype(int)
df['f3'] =  (z.between( 0.01,  1.25, inclusive='both')).astype(int)
df['f4'] =  (z >  1.25).astype(int)

# OLS:  z_t  =  a + b₁f1 + b₂f2 + b₃f3 + b₄f4
res_f = sm.OLS(z, sm.add_constant(df[['f1','f2','f3','f4']]), missing='drop').fit()
for f in ['f1','f2','f3','f4']:
    df[f] *= res_f.params[f]                   #  re-define as β_j·f_j

# ------------------------------------------------------------------
# 4.  Local-projection loop
# ------------------------------------------------------------------
b1, b2, v1, v2 = [], [], [], []                # stores for panels 1 & 2

cvars = ['Industrial Production','Consumer Price Index' , 'Unemployment', 'Fed Funds Rate']
outcome = cvars[0]
cvars.remove(outcome)


for h in range(h_max + 1):
    # dependent variable: y_{t+h} − y_{t−1}
    y = df[outcome].shift(-h) - df[outcome].shift(1)

    # RHS construction
    X = df[['f1', 'f2', 'f3', 'f4']].copy()
    X['y_lag2'] = df[outcome].shift(2)

    # contemporaneous controls: x1_t, x2_t, ...
    for i, var in enumerate(cvars, 1):
        X[f'x{i}_t'] = df[var]

    # 12 lags of first-differences for each control (assume columns d_FFR, d_CPI, d_UN, d_prod)
    for lag in range(1, 13):
        X[f'dx1_lag{lag}'] = df['d_CPI'].shift(lag)     # x1 = CPI
        X[f'dx2_lag{lag}'] = df['d_UN'].shift(lag)      # x2 = UN
        X[f'dx3_lag{lag}'] = df['d_FFR'].shift(lag)     # x3 = FFR
        X[f'dx4_lag{lag}'] = df['d_prod'].shift(lag)    # generic prod control

    # 3–12 lags of y differences
    for lag in range(3, 13):
        X[f'dy_lag{lag}'] = df['d_IP'].shift(lag)  # assumes 'd_IP' is first-diff of outcome

    reg = sm.OLS(y, sm.add_constant(X), missing='drop').fit(cov_type='HC1')

    # extract coefficients and covariances of f-dummies
    b_f1, b_f2, b_f3, b_f4 = reg.params[['f1', 'f2', 'f3', 'f4']]
    vc = reg.cov_params().loc[['f1', 'f2', 'f3', 'f4'], ['f1', 'f2', 'f3', 'f4']]

    # Panel 1: f1 + f3
    b1.append(b_f1 + b_f3)
    v1.append(vc.loc['f1', 'f1'] + vc.loc['f3', 'f3'] + 2 * vc.loc['f1', 'f3'])

    # Panel 2: f2 + f4
    b2.append(b_f2 + b_f4)
    v2.append(vc.loc['f2', 'f2'] + vc.loc['f4', 'f4'] + 2 * vc.loc['f2', 'f4'])


b1, b2, se1, se2 = np.array(b1), np.array(b2), np.sqrt(v1), np.sqrt(v2)

# ------------------------------------------------------------------
# 5.  Plot
# ------------------------------------------------------------------
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6,6), sharex=True)
horiz = np.arange(h_max+1)

for ax, b_hat, se in [(ax1, b1, se1), (ax2, b2, se2)]:
    ax.plot(horiz, b_hat, lw=1.8)
    ax.fill_between(horiz, b_hat - zcrit*se, b_hat + zcrit*se, alpha=0.25)
    ax.axhline(0, lw=0.8, ls='--')
    ax.set_xlim(0, h_max);  ax.set_ylabel(r'ΔIP response')

ax1.set_title(r'$(f_1+f_3)$ impulse')
ax2.set_title(r'$(f_2+f_4)$ impulse')
ax2.set_xlabel('horizon  (months)')

fig.tight_layout()
plt.show()
